In [8]:
import numpy as np
from scipy.stats import levy_stable, uniform, multivariate_normal
from scipy.interpolate import interp1d

def prior_sample():
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    alpha = uniform.rvs(1.1, 0.9)  # U[1.1, 2]
    beta = uniform.rvs(-1, 2)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 600)  # U[-300, 300]
    return alpha, beta, gamma, delta

def simulate_data(theta):
    # Simulate data from the model given parameters theta
    alpha, beta, gamma, delta = theta
    return levy_stable.rvs(alpha, beta, loc=gamma, scale=delta, size=200)

def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

def summary_statistics(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    # For univariate alpha-stable models, use quantile-based estimators along with the mean of the data
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return v_alpha, v_beta, v_gamma, mean_x

def estimate_covariance(theta_hat):
    # Estimate covariance of summary statistics based on 1000 draws from the model
    samples = [simulate_data(theta_hat) for _ in range(1000)]
    summary_stats = np.array([summary_statistics(sample) for sample in samples])
    return np.cov(summary_stats, rowvar=False)

def smoothing_kernel(y, x, epsilon, covariance):
    # Define smoothing kernel using a Gaussian kernel with estimated covariance
    return multivariate_normal.pdf(y, mean=x, cov=epsilon**2 * covariance)

def mutation_kernel(theta_t, theta_prev, weights_prev):
    # Define mutation kernel based on the provided information
    # For univariate alpha-stable models, use a Gaussian kernel with mean theta_prev[i] and covariance Λ
    # Λ = diag(0.25, 0.25, 1, 1)
    covariance = np.diag([0.25, 0.25, 1, 1])
    new_theta = np.zeros_like(theta_t)
    print('new_theta : ', new_theta)
    """
    for i in range(len(weights_prev)):
        print(f'theta_prev[{i}] = {theta_prev[i]}')
        new_theta += weights_prev[i] * multivariate_normal(mean=theta_prev[i], cov=covariance).pdf(theta_t)
    """
    return new_theta
def M_t(theta, theta_prev, weights_prev) :
    covariance = np.diag([0.25, 0.25, 1, 1])
    return np.sum(weights_prev*multivariate_normal(mean=theta_prev, cov=covariance).pdf(theta))
    
def resample(weights):
    # Resampling step
    indexes = np.random.choice(range(len(weights)), size=len(weights), p=weights)
    return indexes

def compute_weights(theta, data, t, ct, epsilon):
    # Compute weights and rejection step
    lf_posterior = likelihood_free_posterior(theta, data, epsilon)
    mutation = mutation_kernel(theta)
    weight, corrected_weight = lf_posterior / mutation, 0 # Corrected weight initially set to 0
    rejection_prob = 1 - min(1, weight / ct)
    if np.random.rand() < rejection_prob:
        return None, None  # Reject sample
    else:
        return weight, weight / (1 - rejection_prob)

def likelihood_free_posterior(theta, data, epsilon):
    # Compute likelihood-free posterior using Monte Carlo estimate
    P = 1000  # Number of Monte Carlo samples
    samples = [simulate_data(theta) for _ in range(P)]
    summary_y = summary_statistics(data)
    weights = [smoothing_kernel(summary_y, summary_statistics(x), epsilon) for x in samples]
    weights /= np.sum(weights)  # Normalize weights
    return np.mean(weights)

def main():
    T = 10  # Number of iterations
    N = 1000  # Number of particles
    epsilons = np.linspace(0.1, 0.01, T)  # Tolerance schedule
    print('espsilons', epsilons)
    # Simulate observed data
    observed_data = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    print('observed data y : \n',observed_data)
    # Initialize weights and particles
    weights = np.ones(N) / N
    print('weights:', weights)
    particles = [prior_sample() for _ in range(N)]
    print("particles : ",particles)
    theta = particles

    for t in range(T):
        print('t=',t)
        epsilon = epsilons[t]
        print("e_t = ", epsilon)
        # Resample particles
        #indexes = resample(weights)
        #print("indexes : ", indexes)
        theta_prev = theta
        
        # Mutation and correction step
        #sample theta from the mutation kernel density estimate
        #for i in range(N):
            #theta_prev = particles[indexes[i]]
        print("theta_prev :\n", theta_prev)
        theta = mutation_kernel(theta_t=theta_prev,theta_prev=theta_prev,weights_prev=weights)
        print('theta :\n', theta)
        print('theta.shape = ',theta.shape )
        weight, corrected_weight = compute_weights(theta, observed_data, t, np.quantile(weights, 0.9), epsilon)
        print("weigt = ",weight)
        if weight is not None:
            particles[i] = theta
            weights[i] = corrected_weight
        print("weights : ", weights)
        print("particles : ", particles)

    # Final weighted sample
    final_particles = [(weights[i], particles[i]) for i in range(N)]



In [9]:
main()

espsilons [0.1  0.09 0.08 0.07 0.06 0.05 0.04 0.03 0.02 0.01]
observed data y : 
 [ 7.16805772e+00  1.49506266e+01 -9.04253016e+00 -4.76654075e+00
  2.15501152e+01  2.58846460e+00 -2.85231146e-02  9.18634530e+00
  1.81740685e+01  2.53739283e+01  2.33768350e+01  2.71476882e+01
  7.82470799e+00  4.65615092e+00  1.03643654e+01  1.00894811e+01
  1.62689112e+01  4.14669581e+01  8.42855546e+00  3.48260059e+00
 -1.83741383e+01 -1.18634400e+01  3.82836274e+00  2.02505513e+01
  6.04406206e+01  1.41495170e+01  3.05698821e+00  8.41283148e+00
  2.62705754e+01  1.15295049e+02  1.74024742e+01  3.78494344e+00
  2.75726313e+00 -1.99267366e+01 -8.20298325e+00 -6.13451619e+00
  7.79628554e+00  8.90599816e+00  7.44077823e+00  9.92720218e+00
  3.59466003e+00 -7.18862172e+00  3.43602430e+01  2.92883221e+02
  1.85500022e+00  1.05142959e+00  9.35269285e+00  9.65607471e+00
  2.70278786e-01  2.40638550e+01  7.21018772e+00  1.82722629e+01
  6.73704178e+01  2.41888173e+01  9.31732438e+00  1.82738390e+01
  5.3857

ValueError: operands could not be broadcast together with shapes (1000,4) (1000,) (1000,4) 

In [ ]:
multivariate_normal(4, )